In [ ]:
import pyes
import json
from collections import defaultdict
import datetime
from bokeh.charts import BoxPlot, Donut, Bar, output_notebook, show
from bokeh.charts.attributes import cat
import pandas as pd

host = '140.118.155.14:9200'


In [ ]:
def getNumOfFans(fid):
    conn = pyes.es.ES(host)
    tq = pyes.query.TermQuery(field="fid", value=fid)
    qsearch = pyes.query.Search(tq)
    result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='fanpage')
    for oneResult in result:
        return oneResult['likes_count']
def getNumOfPosts(fid):
    data = []
    conn = pyes.es.ES(host)
    tq = pyes.query.TermQuery(field="fid", value=fid)
    tagg = pyes.aggs.TermsAgg('post', field= 'fid')
    qsearch = pyes.query.Search(tq)
    qsearch.agg.add(tagg)
    result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='post')
    for post_summary in result.aggs["post"]["buckets"]:
        return post_summary["doc_count"]
def getNumOfTotalLike(fid):
    conn = pyes.es.ES(host)
    tq = pyes.query.TermQuery(field="fid", value=fid)
    tagg = pyes.aggs.SumAgg('like', field= 'likes_count')
    qsearch = pyes.query.Search(tq)
    qsearch.agg.add(tagg)
    result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='post')
    #print json.dumps(result.aggs,indent=2)
    return result.aggs['like']['value']
def getNumOfUniqueLike(fid):
    conn = pyes.es.ES(host)
    tq = pyes.query.TermQuery(field="fid", value=fid)
    tagg = pyes.aggs.TermsAgg('uid', field= 'likes.data.id', size=9999999)
    qsearch = pyes.query.Search(tq)
    qsearch.agg.add(tagg)
    result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='post')
    #print json.dumps(result.aggs,indent=2)
    return len(result.aggs['uid']['buckets'])
def getNumOfUniqueComment(fid):
    conn = pyes.es.ES(host)
    tq = pyes.query.TermQuery(field="fid", value=fid)
    qsearch = pyes.query.Search(tq)
    tagg = pyes.aggs.TermsAgg('uid', field= 'from.id', size=9999999)
    #tagg1 = pyes.aggs.TermsAgg('id', field= 'id')
    #tagg.sub_aggs.append(tagg1)
    qsearch.agg.add(tagg)
    result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='comment')
    #print qsearch.serialize()
    #print json.dumps(result.aggs,indent=2)
    return len(result.aggs["uid"]["buckets"])
def getNumOfShare(fid):
    conn = pyes.es.ES(host)
    tq = pyes.query.TermQuery(field="fid", value=fid)
    qsearch = pyes.query.Search(tq)
    #tagg = pyes.aggs.SumAgg('share', field= 'shares.count')
    tagg = pyes.aggs.ValueCountAgg('share', field='shares.count')
    qsearch.agg.add(tagg)
    result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='post')
    #print qsearch.serialize()
    #print json.dumps(result.aggs,indent=2)
    return result.aggs["share"]["value"]
def getNumOfTag(fid):
    conn = pyes.es.ES(host)
    tq = pyes.query.TermQuery(field="fid", value=fid)
    qsearch = pyes.query.Search(tq)
    tagg = pyes.aggs.ValueCountAgg('tag', field='with_tags.data.name')
    qsearch.agg.add(tagg)
    result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='post')
    #print qsearch.serialize()
    #print json.dumps(result.aggs,indent=2)
    return result.aggs["tag"]["value"]

In [ ]:
fid = ['100489613340306', '304834096975', '177612180793']
fans = []
posts = []
like = []
share = []
tag = []
for oneFid in fid:
    fans.append(getNumOfFans(oneFid))
    posts.append(getNumOfPosts(oneFid))
    like.append(getNumOfUniqueLike(oneFid))
    share.append(getNumOfShare(oneFid))
    tag.append(getNumOfTag(oneFid))
data = defaultdict(list)


In [ ]:
data1 = {
    'value':[i for i in fans],
    'label':['fan', 'fan', 'fan'],
    'group':['Samsoung', 'Sony', 'Asus']
}
data2 = {
    'value':[i for i in posts],
    'label':['post', 'post', 'post'],
    'group':['Samsoung', 'Sony', 'Asus']
}
data3 = {
    'value':[i for i in like],
    'label':['like', 'like', 'like'],
    'group':['Samsoung','Sony', 'Asus']
}
data4 = {
    'value':[i for i in share],
    'label':['share', 'share', 'share'],
    'group':['Samsoung','Sony', 'Asus']
}
data5 = {
    'value':[i for i in tag],
    'label':['tag', 'tag', 'tag'],
    'group':['Samsoung','Sony', 'Asus']
}
print data1,data2,data3, data4, data5
fans_bar1 = Bar(data1, values='value', label='label', group='group', title='fans')
fans_bar2 = Bar(data2, values='value', label='label', group='group', title='post')
fans_bar3 = Bar(data3, values='value', label='label', group='group', title='like')
fans_bar4 = Bar(data4, values='value', label='label', group='group', title='share')
fans_bar5 = Bar(data5, values='value', label='label', group='group', title='tag')
output_notebook()
show(fans_bar1)
show(fans_bar2)
show(fans_bar3)
show(fans_bar4)
show(fans_bar5)

In [ ]:
data1 = {
    'value':[i/j for i, j in zip(fans, posts)],
    'label':['fan', 'fan', 'fan'],
    'group':['Samsoung', 'Sony', 'Asus']
}
data2 = {
    'value':[i for i in posts],
    'label':['post', 'post', 'post'],
    'group':['Samsoung', 'Sony', 'Asus']
}
data3 = {
    'value':[i/j for i, j in zip(like, posts)],
    'label':['like', 'like', 'like'],
    'group':['Samsoung','Sony', 'Asus']
}
data4 = {
    'value':[i/j for i, j in zip(like, posts)],
    'label':['share', 'share', 'share'],
    'group':['Samsoung','Sony', 'Asus']
}
data5 = {
    'value':[i for i in tag],
    'label':['tag', 'tag', 'tag'],
    'group':['Samsoung','Sony', 'Asus']
}
print data1,data2,data3, data4, data5
fans_bar1 = Bar(data1, values='value', label='label', group='group', title='fans_div_post')
fans_bar2 = Bar(data2, values='value', label='label', group='group', title='post')
fans_bar3 = Bar(data3, values='value', label='label', group='group', title='like_div_post')
fans_bar4 = Bar(data4, values='value', label='label', group='group', title='share_div_post')
fans_bar5 = Bar(data5, values='value', label='label', group='group', title='tag')
output_notebook()
show(fans_bar)

In [ ]:
def barCommentPerMonth(fid):
    conn = pyes.es.ES(host)
    tq = pyes.query.TermQuery(field="fid", value=fid)
    DHAgg = pyes.aggs.DateHistogramAgg('month' ,field='created_time', interval='month',sub_aggs=[])
    tagg = pyes.aggs.TermsAgg('pid', field= 'fid_pid')
    DHAgg.sub_aggs.append(tagg) 
    qsearch = pyes.query.Search(tq)
    qsearch.agg.add(DHAgg) 
    result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='comment')
    #top of 100
    #print qsearch.serialize()
    datayear = defaultdict(list)
    #print json.dumps(result.aggs,indent=2) 
    for r in result.aggs["month"]["buckets"]:
        for pid in r["pid"]["buckets"]:
            year=datetime.datetime.strptime(r["key_as_string"], "%Y-%m-%dT%H:%M:%S.%fZ").strftime('%Y')
            month=datetime.datetime.strptime(r["key_as_string"], "%Y-%m-%dT%H:%M:%S.%fZ").strftime('%m')
            month = int(month) - 1
            #print year, month
            if year not in datayear.iterkeys():
                datayear[year] = [0,0,0,0,0,0,0,0,0,0,0,0]
                print datayear
                datayear[year][month] += pid["doc_count"]
            else:
                datayear[year][month] += pid["doc_count"]
    #print datayear
    year = ['2013', '2014', '2015', '2016']
    for oneYear in year:
        dataDraw = {
            'label' : ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"],
            'count' : datayear[oneYear]
        }
        comment = Bar(dataDraw, values='count', label=cat(columns='label', sort=False), title='Comment count per month ' + oneYear)
        output_notebook()
        show(comment)
barCommentPerMonth(fid1)

In [ ]:
def boxPlotCommentPerMonth(fid):
    conn = pyes.es.ES(host)
    tq = pyes.query.TermQuery(field="fid", value=fid)
    DHAgg = pyes.aggs.DateHistogramAgg('month' ,field='created_time', interval='month',sub_aggs=[])
    tagg = pyes.aggs.TermsAgg('pid', field= 'fid_pid')
    DHAgg.sub_aggs.append(tagg) 
    qsearch = pyes.query.Search(tq)
    qsearch.agg.add(DHAgg) 
    result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='comment')
    #top of 100
    #print qsearch.serialize()
    datayear = dict()
    #print json.dumps(result.aggs,indent=2) 
    for r in result.aggs["month"]["buckets"]:
        for pid in r["pid"]["buckets"]:
            year=datetime.datetime.strptime(r["key_as_string"], "%Y-%m-%dT%H:%M:%S.%fZ").strftime('%Y')
            month=datetime.datetime.strptime(r["key_as_string"], "%Y-%m-%dT%H:%M:%S.%fZ").strftime('%m')
            month = int(month)
            #print year, month
            #if mont not in datayear[year].iterkeys():
            if year not in datayear.iterkeys():
                datayear[year] = defaultdict(list)
            datayear[year][month].append(pid["doc_count"])
    #print datayear
    year = ['2013', '2014', '2015', '2016']
    dataDraw = {'label' : [],'count':[]}
    for month in datayear['2013'].iterkeys():
        for data in datayear['2013'][month]:
            if data < 200:
                continue
            dataDraw['label'].append(month)
            dataDraw['count'].append(data)
    #print dataDraw
    comment2013 = BoxPlot(dataDraw, values='count', label='label', title='Comment count per month 2013')
    
    dataDraw = {'label' : [],'count':[]}
    for month in datayear['2013'].iterkeys():
        for data in datayear['2013'][month]:
            if data < 200:
                continue
            dataDraw['label'].append(month)
            dataDraw['count'].append(data)
    #print dataDraw
    comment2014 = BoxPlot(dataDraw, values='count', label='label', title='Comment count per month 2013')
    
    dataDraw = {'label' : [],'count':[]}
    for month in datayear['2014'].iterkeys():
        for data in datayear['2014'][month]:
            if data < 200:
                continue
            dataDraw['label'].append(month)
            dataDraw['count'].append(data)
    #print dataDraw
    comment2014 = BoxPlot(dataDraw, values='count', label='label', title='Comment count per month 2014')
    
    dataDraw = {'label' : [],'count':[]}
    for month in datayear['2015'].iterkeys():
        for data in datayear['2015'][month]:
            if data < 200:
                continue
            dataDraw['label'].append(month)
            dataDraw['count'].append(data)
    #print dataDraw
    comment2015 = BoxPlot(dataDraw, values='count', label='label', title='Comment count per month 2015')
    
    dataDraw = {'label' : [],'count':[]}
    for month in datayear['2016'].iterkeys():
        for data in datayear['2016'][month]:
            if data < 200:
                continue
            dataDraw['label'].append(month)
            dataDraw['count'].append(data)
    #print dataDraw
    comment2016 = BoxPlot(dataDraw, values='count', label='label', title='Comment count per month 2016')
    output_notebook()
    show(comment2013)
    show(comment2014)
    show(comment2015)
    show(comment2016)
boxPlotCommentPerMonth(fid3)

In [ ]:
def boxPlotPostPerMonth(fid): 
    conn = pyes.es.ES(host)
    tq = pyes.query.TermQuery(field="fid", value=fid)
    DHAgg = pyes.aggs.DateHistogramAgg('month' ,field='created_time', interval='month',sub_aggs=[])
    tagg = pyes.aggs.TermsAgg('pid', field= 'fid_pid')
    DHAgg.sub_aggs.append(tagg) 
    qsearch = pyes.query.Search(tq) 
    qsearch.agg.add(DHAgg) 
    result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='comment')
    #print json.dumps(result.aggs,indent=2) 
    data=[]
    for r in result.aggs["month"]["buckets"]:
        for pid in r["pid"]["buckets"]:
            monthyear=datetime.datetime.strptime(r["key_as_string"], "%Y-%m-%dT%H:%M:%S.%fZ").strftime('%Y/%m')
            data.append([pid["doc_count"],monthyear,pid["key"],r["key"]])
    print data
boxPlotPostPerMonth(fid1)